# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import random

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file from the GitHub URL into a Pandas DataFrame
url = "https://raw.githubusercontent.com/nikol-m/python-api-challenge/main/cities.csv"
city_data_df = pd.read_csv(url)

# Display sample data
print(city_data_df.head())


   City_ID       City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0      pevek  69.7008  170.3133    278.74        88         100   
1        1      avera  33.1940  -82.5271    298.18        52         100   
2        2  grindavik  63.8424  -22.4338    287.41        72          75   
3        3     albany  42.6001  -73.9662    303.47        69          43   
4        4     barrow  71.2906 -156.7887    281.16        93         100   

   Wind Speed Country        Date  
0        0.36      RU  1690650521  
1        1.17      US  1690650522  
2        4.63      IS  1690650522  
3        0.45      US  1690650422  
4        4.12      US  1690650523  


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
# Create a map centered at a specific location (e.g., coordinates of the first city in the DataFrame)


# Define a function to map humidity values to colors
map_center = [city_data_df["Lat"].iloc[0], city_data_df["Lng"].iloc[0]]
mymap = folium.Map(location=map_center, zoom_start=5)

# Configure the map plot
def get_color(humidity):
    if humidity <= 30:
        return 'green'
    elif humidity <= 60:
        return 'blue'
    else:
        return 'pink'

# Iterate through the DataFrame and add a marker for each city with size and color based on humidity
for index, row in city_data_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    humidity = row["Humidity"]

    # Create a circle marker with size and color based on humidity
    folium.CircleMarker(
        location=[lat, lng],
        radius=humidity / 8,  
        popup=f"City: {row['City']}, Humidity: {humidity}",
        fill=True,
        fill_color=get_color(humidity),
        color='gray',
        fill_opacity=0.4
    ).add_to(mymap)

# Display the map
mymap

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[(city_data_df['Max Temp'] > 20) & (city_data_df['Humidity'] < 80) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities.head(20)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,ponta do sol,32.6667,-17.1000,300.50,71,0,0.83,PT,1690650540
31,31,carnarvon,-24.8667,113.6333,289.78,69,0,6.72,AU,1690650542
48,48,bardiyah,31.7561,25.0865,300.30,69,0,9.02,LY,1690650555
101,101,ceres,37.5949,-120.9577,303.33,27,0,2.57,US,1690650613
111,111,saint george,37.1041,-113.5841,308.94,25,0,3.09,US,1690650619
114,114,tiznit,29.5833,-9.5000,304.23,37,0,3.64,MA,1690650621
117,117,acapulco,16.8634,-99.8901,305.05,58,0,4.12,MX,1690650623
151,151,luangwa,-15.6167,30.4167,295.97,42,0,1.53,ZM,1690650645
154,154,broome,42.2506,-75.8330,300.94,78,0,4.12,US,1690650647
162,162,east london,-33.0153,27.9116,289.67,67,0,3.60,ZA,1690650651


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
26,26,ponta do sol,32.6667,-17.1000,300.50,71,0,0.83,PT,1690650540,
31,31,carnarvon,-24.8667,113.6333,289.78,69,0,6.72,AU,1690650542,
48,48,bardiyah,31.7561,25.0865,300.30,69,0,9.02,LY,1690650555,
101,101,ceres,37.5949,-120.9577,303.33,27,0,2.57,US,1690650613,
111,111,saint george,37.1041,-113.5841,308.94,25,0,3.09,US,1690650619,
114,114,tiznit,29.5833,-9.5000,304.23,37,0,3.64,MA,1690650621,
117,117,acapulco,16.8634,-99.8901,305.05,58,0,4.12,MX,1690650623,
151,151,luangwa,-15.6167,30.4167,295.97,42,0,1.53,ZM,1690650645,
154,154,broome,42.2506,-75.8330,300.94,78,0,4.12,US,1690650647,
162,162,east london,-33.0153,27.9116,289.67,67,0,3.60,ZA,1690650651,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Assume you already have the hotel_df DataFrame created with columns "City", "Country", "Lat", and "Lng"

# Set the API key and other parameters
api_key = "3092a0917c0247e9800baef5180c702a"
base_url = "https://api.geoapify.com/v2/places"
categories = "accommodation.hotel"

# Function to get the nearest hotel for a given set of coordinates
def get_nearest_hotel(lat, lng):
    # Construct the API request URL with the provided coordinates
    url = f"{base_url}?categories={categories}&apiKey={api_key}&lat={lat}&lon={lng}"

    # Make the API request
    response = requests.get(url)

    # Check the API response
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Get the first hotel from the response (nearest one)
        feature = data.get("features", [])[0]
        properties = feature.get("properties", {})
        hotel_name = properties.get("name", "Unknown")

        return hotel_name
    else:
        # Handle the error
        print(f"Error: {response.json()}")
        return "No hotel found"


# Use the Pandas apply function to add the "Hotel Name" for each row in the DataFrame
hotel_df["Hotel Name"] = hotel_df.apply(lambda row: get_nearest_hotel(row["Lat"], row["Lng"]), axis=1)


# Display the DataFrame with the hotel information
print("hotel_df")

# Display sample data
hotel_df.head(20)


hotel_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
26,26,ponta do sol,32.6667,-17.1000,300.50,71,0,0.83,PT,1690650540,Estalagem Ponta do Sol
31,31,carnarvon,-24.8667,113.6333,289.78,69,0,6.72,AU,1690650542,Monkey Mia Dolphin Resort
48,48,bardiyah,31.7561,25.0865,300.30,69,0,9.02,LY,1690650555,فندق البردي
101,101,ceres,37.5949,-120.9577,303.33,27,0,2.57,US,1690650613,Howard Johnson by Wyndham Modesto Ceres
111,111,saint george,37.1041,-113.5841,308.94,25,0,3.09,US,1690650619,"The Advenire, an Autograph Collection Hotel"
114,114,tiznit,29.5833,-9.5000,304.23,37,0,3.64,MA,1690650621,Kerdous
117,117,acapulco,16.8634,-99.8901,305.05,58,0,4.12,MX,1690650623,Hotel del Valle
151,151,luangwa,-15.6167,30.4167,295.97,42,0,1.53,ZM,1690650645,Redcliff Zambezi Lodge
154,154,broome,42.2506,-75.8330,300.94,78,0,4.12,US,1690650647,Quality Inn & Suites
162,162,east london,-33.0153,27.9116,289.67,67,0,3.60,ZA,1690650651,Unknown


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
# Configure the map plot
# Create a map centered at a specific location (e.g., coordinates of the first city in the DataFrame)
map_center = [hotel_df["Lat"].iloc[0], hotel_df["Lng"].iloc[0]]
mymap = folium.Map(location=map_center, zoom_start=5)

# Create a dictionary to map each unique city to a color
city_to_color = {}

# Function to get a unique color for each city
def get_color(city):
    if city not in city_to_color:
        random.seed(city)  # Seed the random number generator with the city name
        color = "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        city_to_color[city] = color
    return city_to_color[city]

# Iterate through the DataFrame and add a marker for each city with the hotel name in the popup
for index, row in hotel_df.head(20).iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    hotel_name = row["Hotel Name"]
    city_name = row["City"]
    country_name = row["Country"]

    # Create a circle marker with a unique color for each city
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        popup=f"City: {city_name}, Country: {country_name}<br>Hotel: {hotel_name}",
        fill=True,
        fill_color=get_color(city_name),
        color='gray',
        fill_opacity=0.6
    ).add_to(mymap)

# Add the key on the right with each city and its corresponding color
key_html = '<div style="position: fixed; bottom: 50px; right: 50px; z-index: 1000; background-color: white; padding: 10px; border: 1px solid gray;">'
for city, color in city_to_color.items():
    key_html += f'<div style="display: flex; align-items: center;"><div style="width: 15px; height: 15px; background-color: {color}; margin-right: 5px;"></div>{city}</div>'
key_html += '</div>'

mymap.get_root().html.add_child(folium.Element(key_html))

# Display the map
mymap
